In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.linalg import hadamard
from scipy.optimize import minimize
import random
import textwrap
from typing import List, Tuple, Dict, Optional
import unittest

from functions import (
    bailey_welch_schedule,
    get_v_star,
    compute_convolutions,
    calculate_objective_serv_time_lookup,
)

In [2]:
# --- Helper: generate_weighted_list ---
def generate_weighted_list(max_s: int, l: float, i: int) -> Optional[np.ndarray]:
    if not isinstance(max_s, int) or max_s <= 0: return None
    if not isinstance(l, (int, float)) or not (1 <= l <= max_s): return None
    if not isinstance(i, int) or not (0 <= i < max_s): return None
    def objective_fn(x: np.ndarray) -> float:
        return (np.dot(np.arange(1, max_s + 1), x) - l) ** 2
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0})
    bounds = [(0, 1)] * max_s
    initial_guess = np.random.dirichlet(np.ones(max_s))
    try:
        result = minimize(objective_fn, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints, options={'maxiter': 300, 'ftol': 1e-9})
        if not result.success: return None
        optimized_probs = result.x
        optimized_probs[optimized_probs < 0] = 0
        current_sum = np.sum(optimized_probs)
        if not np.isclose(current_sum, 1.0):
            if current_sum > 1e-8: optimized_probs /= current_sum
            else: return None
    except Exception: return None
    first_part_probs = optimized_probs[:i] if i > 0 else np.array([])
    second_part_probs = optimized_probs[i:]
    values = np.zeros(max_s + 1)
    if i > 0: values[1 : i + 1] = np.sort(first_part_probs)
    values[i + 1 : max_s + 1] = np.sort(second_part_probs)[::-1]
    final_sum = np.sum(values[1:])
    if not np.isclose(final_sum, 1.0):
        if final_sum > 1e-8: values[1:] /= final_sum
        else: return None
    return values
  
# --- Helper: Generate evenly distributed patient schedule ---

def generate_evenly_distributed_schedule_intervals(n_patients: int, t_intervals: int) -> List[int]:
    """
    Generates a list representing a schedule, aiming to distribute 'n_patients'
    into 't_intervals' as evenly as possible using round(). A '1' indicates
    a scheduled interval, and '0' an empty one.

    Key Behavior:
    - If n_patients == 0: Returns a list of all zeros of length 't_intervals'.
    - If t_intervals < n_patients (and n_patients > 0): Raises a ValueError.
    - If n_patients == t_intervals (and both > 0): Returns a list of all ones.
    - If n_patients < t_intervals (and both > 0): The function attempts to mark
      'n_patients' distinct intervals using rounding.
      The patient slots are chosen using the formula round(j * t_intervals / n_patients)
      for j from 0 to n_patients-1.
    - **Warning**: Using round() might lead to duplicate indices, potentially
      resulting in fewer than 'n_patients' being scheduled. If exactly
      'n_patients' must be scheduled, the floor-based approach or a
      different algorithm should be used.

    Args:
        n_patients: The number of patients to schedule. Must be a non-negative integer.
        t_intervals: The total number of available time intervals. Must be a non-negative integer.

    Returns:
        A list of integers of length 't_intervals'.

    Raises:
        ValueError: If 'n_patients' or 't_intervals' are not non-negative integers.
        ValueError: If 't_intervals' is less than 'n_patients' (and n_patients > 0).
    """
    # Validate inputs
    if not isinstance(n_patients, int) or n_patients < 0:
        raise ValueError("n_patients must be a non-negative integer.")
    if not isinstance(t_intervals, int) or t_intervals < 0:
        raise ValueError("t_intervals must be a non-negative integer.")

    # Handle n_patients == 0
    if n_patients == 0:
        return [0] * t_intervals

    # At this point, n_patients > 0.

    # Core logic based on comparison of n_patients and t_intervals
    if t_intervals < n_patients:
        raise ValueError(
            f"Cannot schedule {n_patients} patients in only {t_intervals} intervals. "
            "Not enough unique slots available."
        )
    elif n_patients == t_intervals:
        return [1] * t_intervals
    else: # n_patients < t_intervals
        schedule = [0] * t_intervals
        scheduled_count = 0
        for j in range(n_patients):
            # Use round() and ensure the result is an integer for indexing
            index = int(round(j * t_intervals / n_patients))

            # Ensure index stays within bounds (shouldn't happen, but safety first)
            if index >= t_intervals:
                index = t_intervals - 1

            schedule[index] = 1

        # Optional: Add a check/warning if the number of scheduled patients
        # is less than n_patients due to rounding collisions.
        # actual_scheduled = sum(schedule)
        # if actual_scheduled < n_patients:
        #     print(f"Warning: Due to rounding, only {actual_scheduled} out of {n_patients} patients were scheduled.")

        return schedule

# --- Helper: Generate start and tail numbers ---
def generate_start_tail_distribution(N_patients: int, T_intervals) -> List[Tuple[int, int]]:
    if not isinstance(N_patients, int) or N_patients <= 0: return []
    distribution = []
    for i in range(N_patients + 1):
        start = i
        for j in range(N_patients + 1 - i):
                tail = j
                if N_patients - start - tail <= T_intervals - 2: distribution.append((start, tail))
    return distribution

In [3]:
# --- Main computation function ---
N_patients = 24
T_intervals = 48
d_interval_len = 10
max_s_time = 30
l_target_avg_service_time = 14.0
i_sorting_split = 10

v_star_matrix = get_v_star(T_intervals)
s_dist = generate_weighted_list(max_s_time, l_target_avg_service_time, i_sorting_split)
if s_dist is None: 
    raise ValueError("Failed to generate service time distribution.")

def compute_schedules_and_objectives(q_no_show: float, w_weight: float):
    """Compute schedules and objectives for given parameters"""
    
    convolutions_dict = compute_convolutions(s_dist.tolist(), N_patients, q_no_show)
    
    distribution = generate_start_tail_distribution(N_patients, T_intervals)
    schedules = []
    objective_values = []
    
    for start, tail in distribution:
        schedule = [start]
        center = generate_evenly_distributed_schedule_intervals(N_patients - start - tail, T_intervals - 2)
        if center is None: 
            raise ValueError("Failed to generate center schedule.")
        schedule.extend(center)
        schedule.append(tail)
        schedules.append(schedule)
        
        ewt, esp = calculate_objective_serv_time_lookup(schedule, d_interval_len, convolutions_dict)
        objective_value = w_weight * ewt + (1 - w_weight) * esp
        objective_values.append(objective_value)
    
    return schedules, objective_values

# --- Create 3x3 subplot grid ---
q_values = [0.1, 0.15, 0.2]
w_values = [0.1, 0.5, 0.9]

::: {.column-screen-inset}

In [4]:
# Create subplots with 3D scenes
fig = make_subplots(
    rows=3, cols=3,
    specs=[[{'type': 'scatter3d'} for _ in range(3)] for _ in range(3)],
    subplot_titles=[f'q={q:.2f}, w={w:.1f}' for w in w_values for q in q_values],
    horizontal_spacing=0.02,
    vertical_spacing=0.12
)

# Global color scale limits for consistency
global_z_min = float('inf')
global_z_max = float('-inf')

# First pass: compute all data and find global min/max for consistent coloring
all_data = {}
for i, w in enumerate(w_values):
    for j, q in enumerate(q_values):
        schedules, objective_values = compute_schedules_and_objectives(q, w)
        all_data[(i, j)] = (schedules, objective_values)
        global_z_min = min(global_z_min, min(objective_values))
        global_z_max = max(global_z_max, max(objective_values))

# Second pass: create plots
subplot_titles = []
for i, w in enumerate(w_values):
    for j, q in enumerate(q_values):
        schedules, objective_values = all_data[(i, j)]
        
        x = [schedule[0] for schedule in schedules]
        y = [schedule[-1] for schedule in schedules]
        z = objective_values
        
        # Find minimum point
        min_index = z.index(min(z))
        min_obj = z[min_index]
        min_schedule = schedules[min_index]
        
        # Store subtitle info
        subplot_titles.append(f'q={q:.2f}, w={w:.1f}<br><sub>Min: {min_obj:.4f}</sub>')
        
        # Create hover text
        schedules_wrapped = [textwrap.wrap(str(schedule), width=75) for schedule in schedules]
        formatted_schedules = ["<br>".join(schedule_lines) for schedule_lines in schedules_wrapped]
        
        hover_texts = [f"Sch: {f_schedule}, N: {sum(schedule)}<br>Obj: {obj_val:.4f}<br>" 
                      for schedule, f_schedule, obj_val in zip(schedules, formatted_schedules, z)]
        
        # Separate normal and minimum points
        normal_indices = [idx for idx in range(len(z)) if idx != min_index]
        normal_x = [x[idx] for idx in normal_indices]
        normal_y = [y[idx] for idx in normal_indices]
        normal_z = [z[idx] for idx in normal_indices]
        normal_hover = [hover_texts[idx] for idx in normal_indices]
        
        min_x = [x[min_index]]
        min_y = [y[min_index]]
        min_z = [z[min_index]]
        min_hover = [hover_texts[min_index]]
        
        # Add normal points
        fig.add_trace(
            go.Scatter3d(
                x=normal_x,
                y=normal_y,
                z=normal_z,
                mode='markers',
                marker=dict(
                    size=4,
                    color=normal_z,
                    colorscale='Viridis',
                    opacity=0.7,
                    cmin=global_z_min,
                    cmax=global_z_max,
                    colorbar=dict(title='Objective') if i == 0 and j == 2 else None,  # Only show colorbar once
                    showscale=True if i == 0 and j == 2 else False
                ),
                name=f'Other (q={q:.2f}, w={w:.1f})',
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=normal_hover,
                showlegend=False
            ),
            row=i+1, col=j+1
        )
        
        # Add minimum point
        fig.add_trace(
            go.Scatter3d(
                x=min_x,
                y=min_y,
                z=min_z,
                mode='markers',
                marker=dict(
                    size=6,
                    color='red',
                    opacity=1.0
                ),
                name=f'Min (q={q:.2f}, w={w:.1f})',
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=min_hover,
                showlegend=False
            ),
            row=i+1, col=j+1
        )
        
# Update subplot titles with minimum information
for idx, title in enumerate(subplot_titles):
    fig.layout.annotations[idx].text = title
    
# Update layout for better aesthetics
fig.update_layout(
    title=dict(
        text='3D Analysis: Schedule Optimization Across Different Parameters<br><sub>Rows: w (weight), Columns: q (no-show probability)</sub>',
        x=0.5,
        y=0.98,
        font=dict(size=16)
    ),
    height=1200,
    width=1400,
    margin=dict(t=100, b=100, l=50, r=50),
    font=dict(size=10)
)

# Update 3D scene properties for all subplots
for i in range(1, 4):
    for j in range(1, 4):
        scene_name = f'scene{(i-1)*3 + j}' if (i-1)*3 + j > 1 else 'scene'
        fig.update_layout(**{
            scene_name: dict(
                xaxis_title='First element',
                yaxis_title='Last element', 
                zaxis_title='Objective',
                camera=dict(
                    eye=dict(x=-1.5, y=-1.5, z=1.5)  # Negative x,y moves origin to front
                ),
                xaxis=dict(range=[0, None]),  # Start x-axis from 0
                yaxis=dict(range=[0, None])   # Start y-axis from 0
            )
        })

fig.show()

:::

::: {.column-screen-inset}

In [5]:
# --- Parameters and Setup for Figure 2 ---
N_patients = 48  # Update N_patients for the second figure
# T_intervals, d_interval_len, max_s_time, l_target_avg_service_time, i_sorting_split, 
# v_star_matrix, and s_dist are unchanged from Figure 1 setup

q_values_fig2 = [0.1, 0.2]
w_values_fig2 = [0.1, 0.9]

# --- Create 2x2 subplot grid for Figure 2 ---
# Create subplots with 3D scenes
fig2 = make_subplots(
    rows=len(w_values_fig2), cols=len(q_values_fig2),
    specs=[[{'type': 'scatter3d'} for _ in range(len(q_values_fig2))] for _ in range(len(w_values_fig2))],
    # Initial subplot titles (will be updated with min objective later)
    subplot_titles=[f'q={q_val:.2f}, w={w_val:.1f}' for w_val in w_values_fig2 for q_val in q_values_fig2],
    horizontal_spacing=0.02,
    vertical_spacing=0.12  # Adjusted vertical spacing for potentially fewer rows
)

# Global color scale limits for consistency in Figure 2
global_z_min_fig2 = float('inf')
global_z_max_fig2 = float('-inf')

# First pass: compute all data for Figure 2 and find global min/max for consistent coloring
all_data_fig2 = {}
for i, w in enumerate(w_values_fig2):
    for j, q in enumerate(q_values_fig2):
        # compute_schedules_and_objectives will use N_patients=48
        schedules, objective_values = compute_schedules_and_objectives(q, w) 
        all_data_fig2[(i, j)] = (schedules, objective_values)
        if objective_values: # Ensure objective_values is not empty
            global_z_min_fig2 = min(global_z_min_fig2, min(objective_values))
            global_z_max_fig2 = max(global_z_max_fig2, max(objective_values))
        else: # Handle cases where no schedules/objectives might be generated
            # This might indicate an issue or an edge case for these parameters
            print(f"Warning: No objective values for q={q}, w={w}, N_patients={N_patients}")


# If no data was generated at all, global_z_min/max might remain inf/-inf. Set to default.
if global_z_min_fig2 == float('inf'): global_z_min_fig2 = 0
if global_z_max_fig2 == float('-inf'): global_z_max_fig2 = 1


# Second pass: create plots for Figure 2
subplot_titles_with_min_fig2 = [] # To store titles with min objective info
for i, w in enumerate(w_values_fig2):
    for j, q in enumerate(q_values_fig2):
        schedules, objective_values = all_data_fig2[(i, j)]
        
        if not schedules: # Skip if no schedules were generated for this combination
            subplot_titles_with_min_fig2.append(f'q={q:.2f}, w={w:.1f}<br><sub>No data</sub>')
            continue

        x_coords = [schedule[0] for schedule in schedules] # 'x' was 'x_coords' to avoid conflict
        y_coords = [schedule[-1] for schedule in schedules] # 'y' was 'y_coords'
        z_coords = objective_values # 'z' was 'z_coords'
        
        # Find minimum point
        min_index = z_coords.index(min(z_coords))
        min_obj = z_coords[min_index]
        # min_schedule = schedules[min_index] # Not explicitly used in title below, but good for debugging
        
        # Store subtitle info
        subplot_titles_with_min_fig2.append(f'q={q:.2f}, w={w:.1f}<br><sub>Min: {min_obj:.4f}</sub>')
        
        # Create hover text
        schedules_wrapped = [textwrap.wrap(str(schedule), width=75) for schedule in schedules]
        formatted_schedules = ["<br>".join(schedule_lines) for schedule_lines in schedules_wrapped]
        
        hover_texts = [f"Sch: {f_schedule}, N: {sum(schedule)}<br>Obj: {obj_val:.4f}<br>" 
                      for schedule, f_schedule, obj_val in zip(schedules, formatted_schedules, z_coords)]
        
        # Separate normal and minimum points
        normal_indices = [idx for idx in range(len(z_coords)) if idx != min_index]
        normal_x = [x_coords[idx] for idx in normal_indices]
        normal_y = [y_coords[idx] for idx in normal_indices]
        normal_z = [z_coords[idx] for idx in normal_indices]
        normal_hover = [hover_texts[idx] for idx in normal_indices]
        
        min_x = [x_coords[min_index]]
        min_y = [y_coords[min_index]]
        min_z = [z_coords[min_index]]
        min_hover = [hover_texts[min_index]]
        
        # Add normal points
        fig2.add_trace(
            go.Scatter3d(
                x=normal_x,
                y=normal_y,
                z=normal_z,
                mode='markers',
                marker=dict(
                    size=4,
                    color=normal_z,
                    colorscale='Viridis', # You can choose a different colorscale if you like
                    opacity=0.7,
                    cmin=global_z_min_fig2,
                    cmax=global_z_max_fig2,
                    # Show colorbar on the top-right plot of this 2x2 grid
                    colorbar=dict(title='Objective') if i == 0 and j == (len(q_values_fig2) - 1) else None,
                    showscale=True if i == 0 and j == (len(q_values_fig2) - 1) else False
                ),
                name=f'Other (q={q:.2f}, w={w:.1f})', # Name for legend (if shown)
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=normal_hover,
                showlegend=False
            ),
            row=i+1, col=j+1
        )
        
        # Add minimum point
        fig2.add_trace(
            go.Scatter3d(
                x=min_x,
                y=min_y,
                z=min_z,
                mode='markers',
                marker=dict(
                    size=6,
                    color='red',
                    opacity=1.0
                ),
                name=f'Min (q={q:.2f}, w={w:.1f})', # Name for legend (if shown)
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=min_hover,
                showlegend=False
            ),
            row=i+1, col=j+1
        )
        
# Update subplot titles with minimum information for Figure 2
for idx, title_text in enumerate(subplot_titles_with_min_fig2):
    if idx < len(fig2.layout.annotations): # Check to prevent error if a subplot had no data
        fig2.layout.annotations[idx].text = title_text
    
# Update layout for Figure 2 aesthetics
fig2.update_layout(
    title=dict(
        text=f'<b>Figure 2:</b> N_patients={N_patients} - Schedule Optimization<br><sub>Rows: w (weight), Columns: q (no-show probability)</sub>',
        x=0.5,
        y=0.98, # Adjusted y to give a bit more space if title is long
        font=dict(size=16)
    ),
    height=1000, # Adjusted height for a 2x2 grid, can be same as original (1200)
    width=1200,  # Adjusted width for a 2x2 grid, can be same as original (1400)
    margin=dict(t=100, b=100, l=50, r=50), # Increased top margin for potentially longer title
    font=dict(size=10)
)

# Update 3D scene properties for all subplots in Figure 2
num_rows_fig2 = len(w_values_fig2)
num_cols_fig2 = len(q_values_fig2)
for r_loop_idx in range(num_rows_fig2):      # 0-indexed loop for rows
    for c_loop_idx in range(num_cols_fig2):  # 0-indexed loop for columns
        # Plotly scene indices are 1-based (scene, scene2, scene3, ...)
        scene_idx_plotly = r_loop_idx * num_cols_fig2 + c_loop_idx + 1
        scene_name = f'scene{scene_idx_plotly}' if scene_idx_plotly > 1 else 'scene'
        
        fig2.update_layout(**{
            scene_name: dict(
                xaxis_title='First element',
                yaxis_title='Last element', 
                zaxis_title='Objective',
                camera=dict(
                    eye=dict(x=-1.5, y=-1.5, z=1.5)
                ),
                xaxis=dict(range=[0, None]),
                yaxis=dict(range=[0, None])
            )
        })

fig2.show()

:::
